Useful URLs

* Seasons page - http://www.espnfc.com/fifa-world-cup/4/scores?season=2014
* Individual match stats - http://www.espn.com/soccer/matchstats?gameId=383303
* line ups for the game - http://www.espn.com/soccer/lineups?gameId=383303

In [1]:
from selenium import webdriver

In [2]:
import datetime
import time
import pandas as pd

In [122]:
browser = webdriver.Chrome("./chromedriver")

In [123]:
header = ["Date","Team1","Team1_score","Team2","Team2_score", "team1_fouls", "team2_fouls", 
        "team1_yellow_cards", "team2_yellow_cards", "team1_red_cards", "team2_red_cards",
        "team1_offsides", "team2_offsides", "team1_won_corners", "team2_won_corners",
        "team1_saves", "team2_saves", "team1_possession", "team2_possession", 
        "team1_shots", "team2_shots","team1_avg_wins", "team1_avg_draws", "team1_avg_gd",
        "team2_avg_wins", "team2_avg_draws", "team2_avg_gd", "team1_lineup", "team2_lineup"]

In [130]:
world_cup_years = ["1998","1994","1990","1986","1982","1978"]
all_data = []
for world_cup_year in world_cup_years:
    all_data += get_year_stats(world_cup_year)
df = pd.DataFrame(all_data, columns= header)    

19980610
19980611
19980612
19980613
19980614
19980615
19980616
19980617
19980618
19980619
19980620
19980621
19980622
19980623
19980624
19980625
19980626
19980627
19980628
19980629
19980630
19980701
19980702
19980703
19980704
19980705
19980706
19980707
19980708
19980709
19980710
19980711
19980712
19980713
19980714
19980715
19980716
finished_current_year
19940617
19940618
19940619
19940620
19940621
19940622
19940623
19940624
19940625
19940626
19940627
19940628
19940629
19940630
19940701
19940702
19940703
19940704
19940705
19940706
19940707
19940708
19940709
19940710
19940711
19940712
19940713
19940714
19940715
19940716
19940717
19940718
19940719
19940720
19940721
19940722
19940723
finished_current_year
19900608
19900609
19900610
19900611
19900612
19900613
19900614
19900615
19900616
19900617
19900618
19900619
19900620
19900621
19900622
19900623
19900624
19900625
19900626
19900627
19900628
19900629
19900630
19900701
19900702
19900703
19900704
19900705
19900706
19900707
19900708
19900709
19

In [131]:
df = pd.DataFrame(all_data, columns= header)    

In [132]:
df

,Date,Team1,Team1_score,Team2,Team2_score,team1_fouls,team2_fouls,team1_yellow_cards,team2_yellow_cards,team1_red_cards,...,team1_shots,team2_shots,team1_avg_wins,team1_avg_draws,team1_avg_gd,team2_avg_wins,team2_avg_draws,team2_avg_gd,team1_lineup,team2_lineup
0,19980610,Brazil,2,Scotland,1,0,0,2,1,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Taffarel_6 Roberto Carlos_2 Cafu_-- Santos ...,-- James Leighton_-- Colin Calderwood_-- Chris...
1,19980610,Morocco,2,Norway,2,0,0,1,0,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Driss Benzekri_-- Abdelkrim El Hadrioui_-- ...,-- Frode Grodas_-- Dan Eggen _-- Henning Berg...
2,19980611,Cameroon,1,Austria,1,0,0,1,1,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Jacques Songo`o_-- Pierre Njanka _-- Raymo...,-- Michael Konsel_-- Andreas Herzog_-- Anton P...
3,19980611,Italy,2,Chile,2,0,0,2,3,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Gianluca Pagliuca_-- Alessandro Nesta_-- Al...,-- Nelson Antonio Tapia_-- Pedro Reyes_-- Javi...
4,19980612,France,3,South Africa,0,0,0,3,1,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Fabien Barthez_-- Bixente Lizarazu_-- Laure...,-- Hans Vonk_-- Pierre Issa OG_-- Mark Fish_...
5,19980612,Paraguay,0,Bulgaria,0,0,0,0,0,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- José Chilavert_-- Carlos Alberto Gamarra_--...,-- Zdravko Zdravkov_-- Trifon Ivanov_-- Ivaylo...
6,19980612,Saudi Arabia,0,Denmark,1,0,0,1,3,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Mohammed Al Deayea_-- Mohammed Al Khilaiwi_...,-- Mogens Krogh_-- Marc Rieper \n _-- Jan Hei...
7,19980613,Netherlands,0,Belgium,0,0,0,0,2,1,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Edwin van der Sar_-- Aron Winter_-- Frank D...,-- Filip De Wilde_-- Franky Van Der Elst_-- Vi...
8,19980613,South Korea,1,Mexico,3,0,0,1,2,1,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Kim Byung-Ji_-- Hong Myung-Bo_-- Lim Lee_--...,-- Jorge Campos_-- Duilio Davino_-- Claudio Su...
9,19980613,Spain,2,Nigeria,3,0,0,3,1,0,...,0 (0),0 (0),0.6,0.4,1.4,0.8,0.2,2.0,-- Andoni Zubizarreta_-- Fernando Hierro _-- ...,-- Peter Rufai_-- Garba Lawal _-- Celestine B...


In [133]:
df.to_csv("espn_data_230618_part2.csv",index=False)

#### For a given year, iterate through the relevant dates

In [4]:
def get_date(url):
    trim = url.split("?src=com")[0]
    date = trim.split("date/")[1]
    return date

In [5]:
def increment_date(date, delta):
    parsed_date = datetime.datetime.strptime(date,"%Y%m%d")
    new_date = parsed_date + datetime.timedelta(delta)
    new_date_string = str(new_date.year) + pad_date(new_date.month) + pad_date(new_date.day) 
    return new_date_string

In [6]:
def pad_date(day_or_month):
    if len(str(day_or_month)) == 1:
        return "0" + str(day_or_month)
    else:
        return str(day_or_month)

In [128]:
def get_year_stats(world_cup_year):
    browser.get('http://www.espnfc.com/fifa-world-cup/4/scores?season='+world_cup_year)
    date_base_url = "http://global.espn.com/football/scoreboard/_/league/FIFA.WORLD/date/"
    current_url = browser.current_url
    date = get_date(current_url)
    year_data = []
    world_cup_duration = 37
    for i in range(world_cup_duration):
        new_date = increment_date(date, i)
        print(new_date)
        browser.get(date_base_url + new_date)
        time.sleep(1)
        css_selector_match = 'article[class="scoreboard soccer fallback js-show"]'
        games = browser.find_elements_by_css_selector(css_selector_match)
        game_ids = get_game_ids(games)
        for game_id in game_ids:
            try:
                year_data.append(get_match_stats(game_id, new_date))
            except:
                continue
    print("finished_current_year")
    return year_data

#### For a given date, get all games for the day

In [11]:
css_selector_match = 'article[class="scoreboard soccer fallback js-show"]'
games = browser.find_elements_by_css_selector(css_selector_match)

In [19]:
def get_game_ids(games):
    game_ids = []
    for game in games:
        game_ids.append(game.get_attribute("id"))
    return game_ids

In [13]:
game_ids

['383302', '383301', '383300']

#### For a given match, get relevant match statistics

In [23]:
match_stats_base_url = "http://www.espn.com/soccer/matchstats?gameId="
match_stats_lineups_base_url = "http://www.espn.com/soccer/lineups?gameId="

In [109]:
game_id = game_ids[0]

In [11]:
# score
def get_score():
    css_selector_scoreboard = 'div[class="competitors sm-score"]'
    scoreboard = browser.find_element_by_css_selector(css_selector_scoreboard)
    try:
        css_selector_home = 'div[class="team away "]' # mistake made by web developers
        home_scoreboard = scoreboard.find_element_by_css_selector(css_selector_home)
        home_team = home_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
        home_score = int(home_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)
        css_selector_away = 'div[class="team home "]'
        away_scoreboard = scoreboard.find_element_by_css_selector(css_selector_away)
        away_team = away_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
        away_score = int(away_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)
    except:
        css_selector_home = 'div[class="team away tied"]' # mistake made by web developers
        home_scoreboard = scoreboard.find_element_by_css_selector(css_selector_home)
        home_team = home_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
        home_score = int(home_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)
        css_selector_away = 'div[class="team home tied"]'
        away_scoreboard = scoreboard.find_element_by_css_selector(css_selector_away)
        away_team = away_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
        away_score = int(away_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)
    return([home_team,home_score,away_team,away_score])

In [108]:
# form
def get_form(home_team_last_five_games):
    form_stats = []
    for game in home_team_last_five_games:
        team1 = game.find_elements_by_class_name("long-name")[0].text
        team2 = game.find_elements_by_class_name("long-name")[1].text
        score = game.find_element_by_class_name("webview-internal").text
        datetime = game.find_element_by_css_selector('span[data-behavior="date_time"]').get_attribute("data-date")
        game_type = game.find_element_by_class_name("competition").text
        game_dict = {
            "team1": team1,
            "team2": team2,
            "score": score,
            "datetime": datetime,
            "game_type": game_type
        }
        form_stats.append(game_dict)
        
    return form_stats

In [109]:
def get_form_stats():
    id_form_home = "gamepackage-team-form-home"
    css_selector_form_game = 'tr'
    home_team_last_five_games = browser.find_element_by_id(id_form_home).find_element_by_css_selector('tbody').find_elements_by_css_selector(css_selector_form_game)
    home_form_stats = get_form(home_team_last_five_games)
    id_form_away = "gamepackage-team-form-away"
    away_team_last_five_games = browser.find_element_by_id(id_form_away).find_element_by_css_selector('tbody').find_elements_by_css_selector(css_selector_form_game)
    away_form_stats = get_form(away_team_last_five_games)
    return home_form_stats, away_form_stats

In [14]:
# Match Stats
def get_game_stats():
    css_home_team_fouls = 'td[data-home-away="home"][data-stat="foulsCommitted"]'
    css_away_team_fouls = 'td[data-home-away="away"][data-stat="foulsCommitted"]'
    css_home_team_yellow_cards = 'td[data-home-away="home"][data-stat="yellowCards"]'
    css_away_team_yellow_cards = 'td[data-home-away="away"][data-stat="yellowCards"]'
    css_home_team_red_cards = 'td[data-home-away="home"][data-stat="redCards"]'
    css_away_team_red_cards = 'td[data-home-away="away"][data-stat="redCards"]'
    css_home_team_offsides = 'td[data-home-away="home"][data-stat="offsides"]'
    css_away_team_offsides = 'td[data-home-away="away"][data-stat="offsides"]'
    css_home_team_won_corners = 'td[data-home-away="home"][data-stat="wonCorners"]'
    css_away_team_won_corners = 'td[data-home-away="away"][data-stat="wonCorners"]'
    css_home_team_saves = 'td[data-home-away="home"][data-stat="saves"]'
    css_away_team_saves = 'td[data-home-away="away"][data-stat="saves"]'


    home_team_fouls = browser.find_element_by_css_selector(css_home_team_fouls).text
    away_team_fouls = browser.find_element_by_css_selector(css_away_team_fouls).text
    home_team_yellow_cards = browser.find_element_by_css_selector(css_home_team_yellow_cards).text
    away_team_yellow_cards = browser.find_element_by_css_selector(css_away_team_yellow_cards).text
    home_team_red_cards = browser.find_element_by_css_selector(css_home_team_red_cards).text
    away_team_red_cards = browser.find_element_by_css_selector(css_away_team_red_cards).text
    home_team_offsides = browser.find_element_by_css_selector(css_home_team_offsides).text
    away_team_offsides = browser.find_element_by_css_selector(css_away_team_offsides).text
    home_team_won_corners = browser.find_element_by_css_selector(css_home_team_won_corners).text
    away_team_won_corners = browser.find_element_by_css_selector(css_away_team_won_corners).text
    home_team_saves = browser.find_element_by_css_selector(css_home_team_saves).text
    away_team_saves = browser.find_element_by_css_selector(css_away_team_saves).text
    
    return [home_team_fouls, away_team_fouls, home_team_yellow_cards, away_team_yellow_cards, home_team_red_cards,
           away_team_red_cards,home_team_offsides, away_team_offsides, home_team_won_corners, away_team_won_corners,
           home_team_saves, away_team_saves]
#     return {
#         "hteamfouls": home_team_fouls,
#         "ateamfouls": away_team_fouls,
#         "hteamyellowcards": home_team_yellow_cards,
#         "ateamyellowcards": away_team_yellow_cards,
#         "hteamredcards": home_team_red_cards,
#         "ateamredcards": away_team_red_cards,
#         "hteamoffsides": home_team_offsides,
#         "ateamoffsides": away_team_offsides,
#         "hteamwoncorners": home_team_won_corners,
#         "ateamwoncorners": away_team_won_corners,
#         "hteamsaves": home_team_saves,
#         "ateamsaves": away_team_saves
#     }

In [15]:
# Possesions
def get_possession_stats():
    css_home_team_possession = 'span[data-home-away="home"][data-stat="possessionPct"]'
    css_away_team_possession = 'span[data-home-away="away"][data-stat="possessionPct"]'

    home_team_possession = browser.find_element_by_css_selector(css_home_team_possession).text
    away_team_possession = browser.find_element_by_css_selector(css_away_team_possession).text
    
    return [home_team_possession, away_team_possession]

In [16]:
# shots
def get_shot_stats():
    css_home_team_shots = 'span[data-home-away="home"][data-stat="shotsSummary"]'
    css_away_team_shots = 'span[data-home-away="away"][data-stat="shotsSummary"]'

    home_team_shots = browser.find_element_by_css_selector(css_home_team_shots).text
    away_team_shots = browser.find_element_by_css_selector(css_away_team_shots).text
    return [home_team_shots, away_team_shots]


In [17]:
def get_lineup(game_id):
    browser.get(match_stats_lineups_base_url + game_id)
    try:
        team1 = browser.find_elements_by_class_name("table-accordion")[0]
        team1_name = team1.find_element_by_css_selector('caption').text
        team1_players = team1.find_element_by_css_selector('tbody').find_elements_by_css_selector("tr")
        team1_player_names = list(map(lambda x: x.find_element_by_class_name("accordion-item").text, team1_players))
        team2 = browser.find_elements_by_class_name("table-accordion")[1]
        team2_name = team2.find_element_by_css_selector('caption').text
        team2_players = team2.find_element_by_css_selector('tbody').find_elements_by_css_selector("tr")
        team2_player_names = list(map(lambda x: x.find_element_by_class_name("accordion-item").text, team2_players))
        return ["_".join(team1_player_names),"_".join(team2_player_names)]
    except:
        return ["",""]

In [115]:
def get_match_stats(game_id, date):
    match_stats_base_url = "http://www.espn.com/soccer/matchstats?gameId="
    match_stats_lineups_base_url = "http://www.espn.com/soccer/lineups?gameId="
    all_stats = [date]
    # game stats (fouls, red cards etc)
    browser.get(match_stats_base_url + game_id)
    browser.execute_script("window.stop();")
    all_stats += get_score()
    all_stats += get_game_stats()
    all_stats += get_possession_stats()
    all_stats += get_shot_stats()
    # stats from last five games
    home_form_stats, away_form_stats = distill_form_stats(get_form_stats())
    all_stats += home_form_stats
    all_stats += away_form_stats
    # lineup
    all_stats += get_lineup(game_id)
    return all_stats
    # lineup
#     browser.get(match_stats_lineups_base_url + game_id)
#     return match_stats

In [116]:
get_match_stats("383292","hello")

['hello',
 'GER',
 4,
 'POR',
 0,
 '8',
 '11',
 '0',
 '1',
 '0',
 '1',
 '2',
 '1',
 '4',
 '6',
 '4',
 '2',
 '54%',
 '43%',
 '13 (9)',
 '14 (9)',
 0.6,
 0.4,
 1.4,
 0.8,
 0.2,
 2.0,
 '1 Manuel Neuer_5 Mats Hummels  _17 Per Mertesacker_4 Benedikt Höwedes_20 Jérôme Boateng_16 Philipp Lahm_18 Toni Kroos_6 Sami Khedira_13 Thomas Müller  \n  _19 Mario Götze_8 Mesut Özil',
 '12 Rui Patrício_2 Bruno Alves_3 Pepe  _5 Fábio Coentrão_21 João Pereira  _4 Miguel Veloso_16 Raul Meireles_8 João Moutinho_9 Hugo Almeida_7 Cristiano Ronaldo_17 Nani']

#### TODO: Figure out how to store form data

In [50]:
test = get_form_stats()

5
5


In [52]:
list(map(lambda x: x["team1"], test[1])) + list(map(lambda x: x["team2"], test[1]))

['Republic of Ireland',
 'Mexico',
 'Portugal',
 'Portugal',
 'Sweden',
 'Portugal',
 'Portugal',
 'Greece',
 'Cameroon',
 'Portugal']

In [32]:
test[1]

[{'datetime': '2014-06-04T18:30Z',
  'game_type': 'International Friendly',
  'score': '2-1',
  'team1': 'Algeria',
  'team2': 'Romania'},
 {'datetime': '2014-05-31T16:00Z',
  'game_type': 'International Friendly',
  'score': '3-1',
  'team1': 'Algeria',
  'team2': 'Armenia'},
 {'datetime': '2014-03-05T17:00Z',
  'game_type': 'International Friendly',
  'score': '2-0',
  'team1': 'Algeria',
  'team2': 'Slovenia'},
 {'datetime': '2013-11-19T18:15Z',
  'game_type': 'FIFA World Cup Qualifying - CAF',
  'score': '1-0',
  'team1': 'Algeria',
  'team2': 'Burkina Faso'},
 {'datetime': '2013-10-12T16:00Z',
  'game_type': 'FIFA World Cup Qualifying - CAF',
  'score': '3-2',
  'team1': 'Burkina Faso',
  'team2': 'Algeria'}]

In [79]:
test_df = pd.DataFrame(test[1])

In [80]:
test_df

,datetime,game_type,score,team1,team2
0,2014-06-10T23:45Z,International Friendly,1-5,Republic of Ireland,Portugal
1,2014-06-07T00:30Z,International Friendly,0-1,Mexico,Portugal
2,2014-05-31T18:30Z,International Friendly,0-0,Portugal,Greece
3,2014-03-05T20:45Z,International Friendly,5-1,Portugal,Cameroon
4,2013-11-19T19:45Z,FIFA World Cup Qualifying - UEFA,2-3,Sweden,Portugal


In [85]:
# wins
wins = sum(test_df[test_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]) > 0))\
 + sum(test_df[test_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0]) > 0))
# draws
draws = sum(test_df[test_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]) == 0))\
+ sum(test_df[test_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0]) == 0))

In [86]:
wins

4

In [87]:
draws

1

In [81]:
teams = list(test_df.team1.values) + list(test_df.team2.values)
main_team = max(set(teams), key=teams.count)

In [82]:
main_team

'Portugal'

In [83]:
avg_gd = sum(test_df[test_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1])))\
+ sum(test_df[test_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0])))
avg_gd /= test_df.shape[0]

In [84]:
avg_gd

2.0

In [106]:
def get_form_summary(team_df):
    teams = list(team_df.team1.values) + list(team_df.team2.values)
    main_team = max(set(teams), key=teams.count)
    # wins
    avg_wins = sum(team_df[team_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]) > 0))\
    + sum(team_df[team_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0]) > 0))
    # draws
    avg_draws = sum(team_df[team_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]) == 0))\
    + sum(team_df[team_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0]) == 0))
    avg_gd = sum(team_df[team_df.team1 == main_team].score.apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1])))\
    + sum(team_df[team_df.team2 == main_team].score.apply(lambda x: int(x.split("-")[1]) - int(x.split("-")[0])))
    avg_wins /= team_df.shape[0]
    avg_draws /= team_df.shape[0]
    avg_gd /= team_df.shape[0]
    return (avg_wins, avg_draws, avg_gd)

In [102]:
def distill_form_stats(form_stats):
    home_df = pd.DataFrame(test[0])
    away_df = pd.DataFrame(test[1])
    home_stats = get_form_summary(home_df)
    away_stats = get_form_summary(away_df)
    return home_stats, away_stats

In [103]:
distill_form_stats(get_form_stats())

5
5
Germany
Portugal


((0.6, 0.4, 1.4), (0.8, 0.2, 2.0))

In [110]:
home, away = distill_form_stats(get_form_stats())

In [111]:
home

(0.6, 0.4, 1.4)